# Axes

The axes are referenced to the tool/carriage:

X - moves towards/away from the tool (perpendicular to the carriage).

Y - moves along the carriage axis

Z - Vertical positioning, rarely used.

A - Yaw of the robot. I believe this rotates around the delivery point of the fibre (the tip of the final roller). The rotation is around the Z axis.

B - Rotation around Y axis (I think), rarely used.

C - Rotation around X axis (I think), rarely used.

E1 - Rotation of the payout eye. 

E2 - Not used, related to the tape placement head I believe.

E3 - This rotates both chucks (headstock and tailstock) together, in a synchronised fashion. 

E4 - This rotates the tailstock ONLY. For example, if you wish to only rotate the headstock, you would require equivalent +ve E3 and -ve E4 motion

In [32]:
import re
d = 'output/'
fname = "A_40mm_helical_45_10"
ftext = open(d+fname+'.src', "r").read()


In [73]:
P = { "E3":0.0, "E4":0.0 }
def linapply(y, brel):
    for e in y.split(","):
        k, v = e.split()
        if brel: # and k in "AXYZ":
            P[k] += float(v)
        else:
            P[k] = float(v)
            
reforblk = "(?s)LIN \{(.*?)\}\nHALT\n.*?FOR N_CYC=1 to (\d+)\n(.*?)ENDFOR"
forblks = re.findall(reforblk, ftext)
#forblks

In [55]:
Pblocks = [ ]
for bknum, forblk in enumerate(forblks):
    P["E3"] = P["E4"] = 0
    linapply(forblk[0], False)
    Ps = [ P.copy() ]
    #print(P["Y"], forblk[0])
    lins = re.findall("LIN_REL \{(.*?)\}", forblk[2])
    k1 = Ps[-1]
    Ncycles = int(forblk[1])
    for N_CYC in range(Ncycles):
        for lin in lins:
            linapply(lin, True)
            Ps.append(P.copy())
            #assert bknum <= 5 or P.get("E3", 0) < 10000, (bknum, P)
    Pblocks.append({"Ps":Ps, "Ncycles":Ncycles})

In [71]:
res=[]
first = True
for P in Pblocks[0]['Ps']:
    if first:
        res.append("LIN {X %+.3f,Y %+.3f,Z %+.3f,A %+.3f,E1 %+.3f,E3 %+.3f,E4 %+.3f}\n" % \
                   (P['X'], P['Y'], 0.0, P['A'], P['E1'], P['E3'], P['E4']))
        first = False
    else:
        res.append("LIN {X %+.3f,Y %+.3f,Z %+.3f,A %+.3f,E1 %+.3f,E3 %+.3f,E4 %+.3f} C_DIS\n" % \
                       (P['X'], P['Y'], 0.0, P['A'], P['E1'], P['E3'], P['E4']))

In [72]:
fout = open(d+fname+'-LIN.src', "w")
fout.write(open("srcheader-HUMM.txt").read())
fout.write(res[0])
fout.write("HALT\n")
fout.write("ACTION(#HUMM3_START,100,1000,#STOPONERROR,ERROR_CODE)\n")
fout.write("$APO.CDIS=85\n")
fout.write("$VEL_EXTAX[1]=100\n")
for l in res[1:]:
    fout.write(l)
fout.write("HALT\n")
fout.write("END\n");
fout.close()